# 0. Install and Import Dependencies

In [1]:
# !pip install mediapipe opencv-python pandas

In [2]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
import random
warnings.filterwarnings('ignore')

In [3]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose # initiate model pose

# 1. Extract Video to Dataset

In [4]:
# for initiate header
landmarks = ["class"]
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [21]:
# writing header for dataset file
with open('../dataset/ROM_v1.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [6]:
# function to write to csv file
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_world_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)
        with open('../dataset/ROM_v1.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [7]:
# function to write to csv file
def export_rotated_landmark(landmarks, action):
    try:
        keypoints = np.array([[res[0], res[1], res[2], res[3]] for res in landmarks]).flatten().tolist()
        keypoints.insert(0, action)
        with open('../dataset/ROM_v1.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [8]:
# def rotate_x_axis(coords, theta):
#     coord, v = np.array([coords.x, coords.y, coords.z]), coords.visibility

#     rotation_marix = np.array([[1,                         0,                         0],
#                                [0, np.cos(np.deg2rad(theta)),-np.sin(np.deg2rad(theta))],
#                                [0, np.sin(np.deg2rad(theta)), np.cos(np.deg2rad(theta))]])

#     new_coord = np.dot(rotation_marix, coord)

#     return [new_coord[0], new_coord[1], new_coord[2], v]
    

In [9]:
def rotate_y_axis(coords, theta):
    coord, v = np.array([coords.x, coords.y, coords.z]), coords.visibility
    deg2rad = np.deg2rad(theta)

    rotation_marix = np.array([[ np.cos(deg2rad), 0, np.sin(deg2rad)],
                               [     0,           1,          0     ],
                               [-np.sin(deg2rad), 0, np.cos(deg2rad)]])

    new_coord = np.dot(rotation_marix, coord)

    return [new_coord[0], new_coord[1], new_coord[2], v]
    

In [10]:
# def rotate_z_axis(coords, theta):
#     coord, v = np.array([coords.x, coords.y, coords.z]), coords.visibility

#     rotation_marix = np.array([[np.cos(np.deg2rad(theta)), -np.sin(np.deg2rad(theta)),0],
#                                [np.sin(np.deg2rad(theta)), np.cos(np.deg2rad(theta)) ,0],
#                                [0                     , 0                      ,1]])

#     new_coord = np.dot(rotation_marix, coord)

#     return [new_coord[0], new_coord[1], new_coord[2], v]

In [11]:
# def rotate_arbitrary_axis(coords, axis, theta):
#     # extract the coordinates
#     coord, v = np.array([coords.x, coords.y, coords.z]), coords.visibility 

#     # change degree to radian
#     deg2rad = np.deg2rad(theta)

#     a = np.cos(deg2rad / 2.0) 
#     b, c, d = - axis * np.sin(deg2rad / 2.0)
    
#     aa, bb, cc, dd = a * a, b * b, c * c, d * d
#     bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    
#     rotation_matrix = np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
#                                 [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
#                                 [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
    
#     new_coord = np.dot(rotation_matrix, coord)

#     # new_coord = (np.cos(deg2rad))*coord + (1-np.cos(deg2rad))*(axis*coord)*axis + (np.sin(deg2rad))*(axis * coord)

#     return [new_coord[0], new_coord[1], new_coord[2], v]

In [12]:
def rotate_landmark(landmarks, range_angle=30) :
    landmark_point = []

    theta = random.randrange(range_angle, 360, range_angle)
    
    for landmark in landmarks.landmark:
        landmark_point.append(rotate_y_axis(landmark, theta))
    
    landmark_point = np.array(landmark_point)

    return landmark_point

In [13]:
def plot_world_landmarks(
    plt,
    ax,
    landmarks,
    visibility_th=0.5,
):
    landmark_point = []

    # for index, landmark in enumerate(landmarks.landmark):
    #     landmark_point.append(
    #         [landmark.visibility, (landmark.x, landmark.y, landmark.z)])
    for index, landmark in enumerate(landmarks):
        landmark_point.append(
            [landmark[3], (landmark[0], landmark[1], landmark[2])])
        
    face_index_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    right_arm_index_list = [11, 13, 15, 17, 19, 21]
    left_arm_index_list = [12, 14, 16, 18, 20, 22]
    right_body_side_index_list = [11, 23, 25, 27, 29, 31]
    left_body_side_index_list = [12, 24, 26, 28, 30, 32]
    shoulder_index_list = [11, 12]
    waist_index_list = [23, 24]

    # 顔
    face_x, face_y, face_z = [], [], []
    for index in face_index_list:
        point = landmark_point[index][1]
        face_x.append(point[0])
        face_y.append(point[2])
        face_z.append(point[1] * (-1))

    # 右腕
    right_arm_x, right_arm_y, right_arm_z = [], [], []
    for index in right_arm_index_list:
        point = landmark_point[index][1]
        right_arm_x.append(point[0])
        right_arm_y.append(point[2])
        right_arm_z.append(point[1] * (-1))

    # 左腕
    left_arm_x, left_arm_y, left_arm_z = [], [], []
    for index in left_arm_index_list:
        point = landmark_point[index][1]
        left_arm_x.append(point[0])
        left_arm_y.append(point[2])
        left_arm_z.append(point[1] * (-1))

    # 右半身
    right_body_side_x, right_body_side_y, right_body_side_z = [], [], []
    for index in right_body_side_index_list:
        point = landmark_point[index][1]
        right_body_side_x.append(point[0])
        right_body_side_y.append(point[2])
        right_body_side_z.append(point[1] * (-1))

    # 左半身
    left_body_side_x, left_body_side_y, left_body_side_z = [], [], []
    for index in left_body_side_index_list:
        point = landmark_point[index][1]
        left_body_side_x.append(point[0])
        left_body_side_y.append(point[2])
        left_body_side_z.append(point[1] * (-1))

    # 肩
    shoulder_x, shoulder_y, shoulder_z = [], [], []
    for index in shoulder_index_list:
        point = landmark_point[index][1]
        shoulder_x.append(point[0])
        shoulder_y.append(point[2])
        shoulder_z.append(point[1] * (-1))

    # 腰
    waist_x, waist_y, waist_z = [], [], []
    for index in waist_index_list:
        point = landmark_point[index][1]
        waist_x.append(point[0])
        waist_y.append(point[2])
        waist_z.append(point[1] * (-1))
            
    ax.cla()
    ax.set_xlim3d(-1, 1)
    ax.set_ylim3d(-1, 1)
    ax.set_zlim3d(-1, 1)

    ax.scatter(face_x, face_y, face_z)
    ax.plot(right_arm_x, right_arm_y, right_arm_z)
    ax.plot(left_arm_x, left_arm_y, left_arm_z)
    ax.plot(right_body_side_x, right_body_side_y, right_body_side_z)
    ax.plot(left_body_side_x, left_body_side_y, left_body_side_z)
    ax.plot(shoulder_x, shoulder_y, shoulder_z)
    ax.plot(waist_x, waist_y, waist_z)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
    plt.pause(.001)

    return

In [ ]:
vid_path = "../../data/ready/perfect/reguler/10.mp4" # path to video
cap = cv2.VideoCapture(vid_path) #initiate video capture

random_angle = 30

#Initiate Holistic Model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    # streaming the video
    while cap.isOpened():
        ret, frame = cap.read()

        # recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # mirror frame
        image_mirror = cv2.flip(frame, 1)
        image_mirror = cv2.cvtColor(image_mirror, cv2.COLOR_BGR2RGB)
        image_mirror.flags.writeable = False
    
        # Make detections
        results = pose.process(image)
        results_mirror = pose.process(image_mirror)
    
        # Recolor image back to BGR for rendering
        image.flags.writeable = False
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        if results.pose_landmarks and results_mirror.pose_landmarks:
            new_landmark = rotate_landmark(results.pose_world_landmarks, random_angle)
            new_landmark_mirror = rotate_landmark(results_mirror.pose_world_landmarks, random_angle)
        

            # extract landmarks
            k = cv2.waitKey(1000)
            if k == 106: # while j is pressed, landmark will be recorded as class 0 => not using momentum
                export_landmark(results, 0)
                export_landmark(results_mirror, 0)
                export_rotated_landmark(new_landmark, 0)
                export_rotated_landmark(new_landmark_mirror, 0)
    
            if k == 107: # while k is pressed, landmark will be recorded as class 1 => using momentum
                export_landmark(results, 1)
                export_landmark(results_mirror, 1)
                export_rotated_landmark(new_landmark, 1)
                export_rotated_landmark(new_landmark_mirror, 1)
        
            if k == 108:
                pass
            
        # Stream vid result
        cv2.imshow("Raw Cam Feed", image)
        # press 'q' than the video will stop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 